In [1]:
from pyspark.sql import SparkSession
spark=SparkSession.builder.master('local').appName('zero_analyst').getOrCreate()
spark

# MySQL connection properties
mysql_host = "localhost"
mysql_port = 3306
mysql_database = "zero_analyst"
mysql_username = "root"
mysql_password = "mypassword"
table_name='retail_sales'

# JDBC URL
jdbc_url = f"jdbc:mysql://{mysql_host}:{mysql_port}/{mysql_database}"
connection_properties = {
    "user": mysql_username,
    "password": mysql_password,
    "driver": "com.mysql.cj.jdbc.Driver"
}



In [2]:
# 01 Write the SQL query to find the second highest salary   SELECT * FROM employees01 
employees01 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'employees01').option("user", mysql_username).option("password", mysql_password).load()

employees01.show()

from pyspark.sql.window import Window
from pyspark.sql.functions import col,dense_rank

department_window=Window.orderBy(col("salary").desc())
second_salary=employees01.withColumn("drnk",dense_rank().over(department_window)).filter(col("drnk")==2).drop(col('drnk'))
second_salary.show()


+-----------+---------------+-----------+--------+
|employee_id|           name| department|  salary|
+-----------+---------------+-----------+--------+
|          1|       John Doe|Engineering|63000.00|
|          2|     Jane Smith|Engineering|55000.00|
|          3|Michael Johnson|Engineering|64000.00|
|          4|    Emily Davis|  Marketing|58000.00|
|          5|    Chris Brown|  Marketing|56000.00|
|          6|    Emma Wilson|  Marketing|59000.00|
|          7|       Alex Lee|      Sales|58000.00|
|          8|    Sarah Adams|      Sales|58000.00|
|          9|     Ryan Clark|      Sales|61000.00|
|         11|           zara|         it|63000.00|
+-----------+---------------+-----------+--------+

+-----------+--------+-----------+--------+
|employee_id|    name| department|  salary|
+-----------+--------+-----------+--------+
|          1|John Doe|Engineering|63000.00|
|         11|    zara|         it|63000.00|
+-----------+--------+-----------+--------+



In [6]:
# Department-wise top 2 highest salaries
dept_rank=Window.partitionBy(col("department")).orderBy(col("salary").desc())
second_dept_salary=employees01.withColumn("drnk",dense_rank().over(dept_rank)).filter(col("drnk")==2).drop(col("drnk"))
second_dept_salary.show()

+-----------+-----------+-----------+--------+
|employee_id|       name| department|  salary|
+-----------+-----------+-----------+--------+
|          1|   John Doe|Engineering|63000.00|
|          4|Emily Davis|  Marketing|58000.00|
|          7|   Alex Lee|      Sales|58000.00|
|          8|Sarah Adams|      Sales|58000.00|
+-----------+-----------+-----------+--------+



In [11]:
# more than avg salary
from pyspark.sql.functions import avg
dept_avg_salary=employees01.groupBy(col("department")).agg(avg(col("salary")).alias("avg_salary"))
dept_avg_salary.show()

employees01.join(dept_avg_salary,on="department",how="inner").filter(col("salary")>col("avg_salary")).show()


+-----------+------------+
| department|  avg_salary|
+-----------+------------+
|      Sales|59000.000000|
|Engineering|60666.666667|
|         it|63000.000000|
|  Marketing|57666.666667|
+-----------+------------+

+-----------+-----------+---------------+--------+------------+
| department|employee_id|           name|  salary|  avg_salary|
+-----------+-----------+---------------+--------+------------+
|      Sales|          9|     Ryan Clark|61000.00|59000.000000|
|Engineering|          1|       John Doe|63000.00|60666.666667|
|Engineering|          3|Michael Johnson|64000.00|60666.666667|
|  Marketing|          4|    Emily Davis|58000.00|57666.666667|
|  Marketing|          6|    Emma Wilson|59000.00|57666.666667|
+-----------+-----------+---------------+--------+------------+



### second question

In [12]:
orders02 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'orders02').option("user", mysql_username).option("password", mysql_password).load()

returns02 = spark.read.format("jdbc").option("url", jdbc_url).option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("dbtable", 'returns02').option("user", mysql_username).option("password", mysql_password).load()

In [ ]:
# 1️⃣ Total number of returned orders per month

